In [ ]:
# default_exp dlc_importer

# DLCImporter

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
#export
import pandas as pd

class DLCImporter:
    """Used to import DLC result files."""
    
    def import_hdf(self, file):
        """Import the specified file."""
        df = pd.read_hdf(file)
        df.columns = df.columns.droplevel(0) # drop redundant scorer
        columns_to_drop = ['a1', 'a2', 'a3', 'a4', 'b1', 'b2', 'b3', 'b4']
        df = df.drop(columns_to_drop, axis=1, level=0)
        df.columns = df.columns.remove_unused_levels()
        return df

In [ ]:
show_doc(DLCImporter.import_hdf)

<h4 id="DLCImporter.import_hdf" class="doc_header"><code>DLCImporter.import_hdf</code><a href="__main__.py#L7" class="source_link" style="float:right">[source]</a></h4>

> <code>DLCImporter.import_hdf</code>(**`file`**)

Import the specified file.

`DLCImporter` can be used to import result files from DeepLabCut in H5 format.
It will automatically drop the unnecessary `scorer` level.

In [ ]:
imp = DLCImporter()
df = imp.import_hdf('example_data/coordinates.h5')
display(df)

For further processing and analyzes, it can be useful to transform the tracked coordinates into relative coordinates (being relative to a specified bodypart). This can be done by specifying a bodypart as the relative origin.


In [ ]:
#export
def transform_to_relative(df, bodypart):
    df_rel = df.copy()

    for bp in df.columns.levels[0]:
        df_rel[bp, 'x'] = df[bp, 'x'] - df[bodypart, 'x']
        df_rel[bp, 'y'] = df[bp, 'y'] - df[bodypart, 'y']

    return df_rel

In [ ]:
df_relative = transform_to_relative(df, 'body')

test_close(df_relative['head','x'][0], -30.6, 0.1)
test_close(df_relative['head','y'][0], -119.1, 0.1)
test_close(df_relative['tail','x'][0], 70.87, 0.1)
test_close(df_relative['tail','y'][0], 135.08, 0.1)

By using the `arctan2` function of Numpy, we are able to calculate the angle of the vector from the origin to a specific point.

In [ ]:
import numpy as np

y = np.array([1, 1, 1, -1, 0.75])
x = np.array([0, 1, -1, 1, 1.5])

result = np.degrees(np.arctan2(y, x))
expected = np.array([90, 45, 135, -45, 26.565051])

np.testing.assert_almost_equal(result, expected, decimal=3)

In [ ]:
# export
def add_middle_neck(df):
    df_middle_neck = df.copy()
    df_middle_neck['middle_neck', 'x'] = (df_middle_neck['left_neck','x'] + df_middle_neck['right_neck', 'x']) / 2
    df_middle_neck['middle_neck', 'y'] = (df_middle_neck['left_neck','y'] + df_middle_neck['right_neck', 'y']) / 2
    return df_middle_neck

Since we already moved the body into the origin, we can now decide for a good point for determining the angle of the pigeon rotation.
The vector between the body and middle point between left_neck and right_neck seems to be suitable.
Therefore, we first calculate this point, we can later use to determine the rotation of the pigeon.

In [ ]:
# assume we have pandas Dataframe
df_example = df.copy()
df_example['left_neck', 'x'][0] = 1
df_example['left_neck', 'y'][0] = 1
df_example['right_neck', 'x'][0] = 2
df_example['right_neck', 'y'][0] = 0.5

df_result = add_middle_neck(df_example)

test_close(df_result['middle_neck','x'][0], 1.5, 0.1)
test_close(df_result['middle_neck','y'][0], 0.75, 0.1)


Based on the `middle_neck` coordinates, we now add the corresponding rotation to the dataframe.

In [ ]:
# export
import numpy as np

def add_rotation(df):
    df = df.copy()
    df['rotation_angle'] = np.degrees(np.arctan2(df['middle_neck', 'y'], df['middle_neck', 'x']))
    return df

In [ ]:
df_result = add_rotation(df_result)
test_close(df_result['rotation_angle'][0], 26.565051, 0.1)

In [ ]:
# export
def apply_rotation(df):
    df = df.copy()
    df.apply(_rotate_row, axis=1)
    return df

def _rotate_row(x):
    theta = np.radians(x['rotation_angle'][0] - 90)

    body_parts = list(x.index.levels[0])
    body_parts.remove('rotation_angle')

    for b in body_parts:
        x1, y = x[b,'x'], x[b,'y']

        c, s = np.cos(theta), np.sin(theta)
        rot = np.matrix([[c, s], [-s, c]])

        rotated = np.dot(rot, [x1, y])

        x[b, 'x'] = rotated[0, 0]
        x[b, 'y'] = rotated[0, 1]

    return x

Finally, we can rotate all relative coordinates by this angle around the origin (which is the `body`), so we stabilize (and normalize) the pigeon orientation.

In [ ]:
df_example = df_result.copy()
df_rotation_applied = apply_rotation(df_example)

test_close(df_rotation_applied['middle_neck','x'][0], 0, 0.1)
test_close(df_rotation_applied['middle_neck','y'][0], 1.6, 0.1)
test_close(df_rotation_applied['left_neck','x'][0], -0.5, 0.1)
test_close(df_rotation_applied['left_neck','y'][0], 1.34, 0.1)

In [ ]:
df_rotation_applied